## Hybrid Crytograph Project – Group 3

Names and students numbers: ________________________

In this project, we will design a hybrid cryptography system using the RSA protocol for key exchange and the Vigenère cipher for bulk data transfer. We will then improve the security of your system by adding an extra layer of random encoding. Lastly, we will complete one of the extensions, from either section 5 (breaking the Vigenère cipher), section 6 (cracking the RSA protocol), or section 7 (implementing RSA+AES and DH+AES).

###Global variables and reusable functions
Early on in the project we realised that planning ahead and making functions reusable would be essential to a successful project as it is more efficient, can reduce time spent coding similar functions that we have already made and also it can create a more robust solution as the functions we use are well tested.

To start we defined some of the global variables that we will use throughout the project.

In [2]:
#Global variables
global alphabet_upper
alphabet_upper="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
global alphabet_lower
alphabet_lower=alphabet_upper.lower()

As we are working with alphabetical messages and sometime keys in this project we decided to write a function clean_text(). The purpose of this is to only keep alphabetical characters from the inputted text. It has parameters of the text, preserve_case and verbose. preserve_case is a parameter that if False converts the message to upper case, this makes it reusable for more questions later on.

In [3]:
#input cleaning function

def clean_text(g_in_text,preserve_case=True,verbose=False):
    in_text=g_in_text#make a local copy
    out_text=""
    if verbose:
        print("Input text is: ", in_text)
    for i in range (0,len(in_text)): #run through all characters and add them if they are in the alphabet
        if in_text[i] in alphabet_lower:
            out_text+=in_text[i]
        elif in_text[i] in alphabet_upper:
            out_text+=in_text[i]
    if verbose:
        print("Text without spaces, symbols, numbers ",out_text)
    if not(preserve_case): #if we aren't preserving the case then we convert it all to upper case
        out_text=str(out_text.upper())
        if verbose:
            print("Case not preserved so convert it to uppercase ",out_text)
    return out_text

It works by iterating through each character in in_text and adding concatenating it to out_text if its in the alphabet. You can try it below.

In [13]:
message="James walked his dog Jeremery!"
print("The cleaned version of that message is: ",clean_text(message,True))

The cleaned version of that message is:  JameswalkedhisdogJeremery


#### Core section 1: Implementing functions to encrypt and decrypt messages using the Caesar cipher and the Vigenère cipher. 

First we wrote the function caesar_cipher(). This is one function to both encrypt and  decrypt messages depending on the argument passed in by the user, this  is to improve space  effiency and increase robustness. Its parameters are g_input_text where g stands for given, g_key,encrypt and verbose.

In [5]:
def caesar_cipher(g_input_text,g_key,encrypt=True,verbose=False):

    #input_text should be fed in as a string, key should be any integer although it only is unique from 0,26

    input_text=g_input_text #creating local copies of arguments
    
    output_text=""
    if encrypt:
        key_multiplier=1
    else:
        key_multiplier=-1
    key=g_key*key_multiplier # g for given

    input_text=clean_text(input_text) #clean the input

    for i in range (0,len(input_text)):
        if input_text[i] in alphabet_upper:
            position_in_alphabet=alphabet_upper.index(input_text[i])
            output_text+=alphabet_upper[(position_in_alphabet+key)%26]
        else: #if adding support for spaces later add it below and change the clean_text function to suppport this
            position_in_alphabet=alphabet_lower.index(input_text[i])
            output_text+=alphabet_lower[(position_in_alphabet+key)%26]
    return output_text

It works in a few simple steps:
1. Make copies of input text and key to make sure they are taken as copies.
2. If the user has specified to encrypt the message then our key is  multiplied by one, otherwise it is multiplied by -1 which means when the shift is applied it is in the opposite direction effectively decrypting the message.
3. We  iterate through the input message and for each character we select the appropriate case of alphabet and we add the key to its numerical position in the alphabet. We take the mod of this to make sure it lies betweeen 0 and 26 then we add the character at that position to the output.
4. We output the output_text

See examples below on how it works on different messages and with the null shift.

In [ ]:
#first we see how it works for a normal key
print(message, "encrypted with key=5 is: ")
print(caesar_cipher(message,5))

#show that the decrypting works
print("We can decrypt this by using the same key: ")
print(caesar_cipher(caesar_cipher(message,5), 5,False))

#Show that it works for the null shift
print("The caesar cipher function also works for the null shift (key=0), this outputs the same message as inputted \nalbeit cleaned by the clean text function")
print(caesar_cipher(message,0))

James walked his dog Jeremery! encrypted with key 5 is: 
OfrjxbfqpjimnxitlOjwjrjwd
We can decrypt this by using the same key: 
JameswalkedhisdogJeremery
The caesar cipher function also works for the null shift (key=0), this outputs the same message as inputted 
albeit cleaned by the clean text function
JameswalkedhisdogJeremery


Next was Vigenere cipher. We named the function v_cipher() as it is clearly distinguishable from caesar_cipher() and hard to misspell. Once again this is only one function to both encrypt and decrypt. It uses caesar cipher to shift each character in the message by an amount given in the key. 

This is our first demonstration of reusable functions. It is more space efficient and more robust as if we modify the functionality of the reusable function it changes for every part of the code it is used, otherwise we would have to change many parts which can cause errors. 

The inputs are g_input_text which is given input text passed in as a string, g_key which is given key also passed in as a string of any length, encrypt which is a boolean to decide if the function is encrypting or decrypting, verbose which is a boolean used for error checking.

In [16]:
#vigenere cipher

def v_cipher(g_input_text,g_key,encrypt=True,verbose=False):
    input_text=clean_text(g_input_text)
    key=clean_text(g_key,preserve_case=False)
    output_text=""

    #for each letter in inpur text message we perform a caesar cipher on it using the ith char in key
    for i in range (0,len(input_text)):
        ith_key=alphabet_upper.index(key[i%len(key)])
        if verbose:
            print("Ith key is", ith_key)
        output_text+=caesar_cipher(input_text[i],ith_key,encrypt)
    return output_text

This function works in a few simple steps. 
1. We make copies of the input text and key and clean them both. Cleaning the key is really helpful here as it means the function can consistently decrypt texts even if someone drops a case in one letter of the key. Note we capitalise the key in the clean_text function by setting preserve_case=False.
2. We iterate through each character in the input text. For each character we cipher it using the part of the key in the corresponding place. Each character in the key is equivalent to an integer that we can encrypt with (with A being the null shift 0). If the message is longer than the key then we start from the beginning of the key again hence the modulus in the line defining ith_key.
3. Using the ith_key we encrypt each character in input_text and concatenate it onto output_text
4. We return output_text

Below are some tests to show the functionality and robustness of this function.

In [19]:
#First show basic functionality of the function
print("Encrypting our standard message with key='HANDS' we get: ")
print(v_cipher(message,"HANDS"))
print("Decrypting this with the same key gets us: ")
print(v_cipher(v_cipher(message,"HANDS"),"HANDS",False))

#Testing varied key inputs
print("We can see that the key 'AAA' is the null cipher: ")
print(v_cipher(message,"AAA"))

print("We can introduce different cases and symbols to our key and see it still works")
print("Key= 'Ha2Nd s'")
print(v_cipher(message,"Ha2Nd s"))
print(v_cipher(v_cipher(message,"Ha2Nd s"),"Ha2Nd s",False))

print("We can encypt with a key longer than the message ")
print(v_cipher(message,"Harry went for a walk with his cat Harriot and stopped at the park"))

#Add random tests and writing to and from files

Encrypting our standard message with key='HANDS' we get: 
QazhkdaynwkhvvvvgWhjlmruq
Decrypting this with the same key gets us: 
JameswalkedhisdogJeremery
We can see that the key 'AAA' is the null cipher: 
JameswalkedhisdogJeremery
We can introduce different cases and symbols to our key and see it still works
Key= 'Ha2Nd s'
QazhkdaynwkhvvvvgWhjlmruq
JameswalkedhisdogJeremery
We can encypt with a key longer than the message 
QadvqseydjryiodzqFmkltmja


When comparing these two approaches, we realised...

#### Core section 2: Implementing a function to systematically break the Caesar cipher using letter frequency analysis.

To write this function, we started by...

In [9]:
print('This is a python block')

This is a python block


#### Core section 3: Writing functions that implement the Hybrid System described below:

"Hybrid System. Alice generates her private and public key. Bob generates a Vigenère key and Vigenère encrypts/enciphers his message with this key. Then, after slicing it into parts (if necessary) he encodes and RSA encrypts his Vigenère key using Alice’s public key
and finally sends both the resulting tuple of ciphertext integers and his Vigenère encrypted message to Alice. Alice uses her private key to RSA decrypt the tuple of ciphertext integers. She then converts/decodes the resulting integers to strings and so reconstructs the Vigenère key. She uses this to Vigenère decrypt/decipher Bob’s message."

To write these functions, we started by...

In [10]:
print('This is a python block')

This is a python block


#### Core section 4: Redesigning our system by performing a random encoding of each letter of the alphabet in to one or more 2-grams.

There are 26 ·25 = 650 many 2-grams made up of distinct letters. The goal is to do encode the alphabet by these 2-grams in such a way that the frequency of occurrence of each letter is disguised.

To implement this encoding, we started by...

In [11]:
print('This is a python block')

This is a python block


#### Extension: Either section 5 (breaking the Vigenère cipher), section 6 (cracking the RSA protocol), or section 7 (implementing RSA+AES and DH+AES)

We decided to go for extension number X because...

We started by...

In [12]:
print('This is a python block')

This is a python block


#### Reflections on the project

In conclusion, through this project we found that...